# Where should I live this year?

## Introduction
I am a bit of a nomad.  I have moved from place to place roughly every 6 months for the last 10 years.  I am a "professional" (I use the term loosely as I only recieve parts & gear, but no paycheck) mountain biker.  My life more or less revolves around the sport.  As such, the majority of the places I have moved to have been because there is an abundance of excellent mountain bike trails in the area.  There are two major areas that I have spent more time at than most others.  They are Southwest Utah, and Whistler, British Colombia.  Both places are incredible.  Southern Utah is incredible in the winter due to the mild weather and lack of snow, while Whistler is the place to be in the summer due to the Whistler Bike Park (it's by far the largest bike park in the world).

Unfortunately, the world is currently in a complete shutdown due to the COVID-19 pandemic.  As much as I would love to return to Whistler in the summer, I am unsure if the bikepark will be open, so I would like to explore other options in British Colombia.

## Data

I hope to use Data Science in order to help me find a solution to the problem, "Where should I live this year?".  I will be leveraging the Foursquare API, as well as the Trailforks API in order to gather data that I can use for my analysis.  Foursquare has a wonderful database of location data, and Trailforks has a database of trail data.  I will try to pin point a few possible neighborhoods that I would like to live at based off of location data of avaiable nearby amenities, nearby hospitals (mountainbiking is quite dangerous), distance to major airports (I love to travel), and of course, how many high quailty trails are in the area.

In [327]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import mechanize
import http.cookiejar
import time
import random
import requests

## Web Scraping
I tried getting access to trailforks API, but they unfortunately did not respond to my email.  I instead built a webscraper to at least get the most important data.  I set my search query to only return the number of Black Diamond, Double Black Diamond, and Pro lines.  These are the types of trails that I enjoy the most.  Some areas may have many many more trails, but if they are only beginner style trails, then I won't be very interested!

In [4]:
# Browser
br = mechanize.Browser()

# Cookie Jar
cj = http.cookiejar.LWPCookieJar()
br.set_cookiejar(cj)

# Browser options
br.set_handle_equiv(True)
br.set_handle_gzip(True)
br.set_handle_redirect(True)
br.set_handle_referer(True)
br.set_handle_robots(False)
br.set_handle_refresh(mechanize._http.HTTPRefreshProcessor(), max_time=1)

br.addheaders = [('User-agent', 'Chrome')]

br.open('https://www.trailforks.com/login/')

br.select_form(nr=1)

# User credentials
br.form['username-login-loginlen'] = 'JessieJ'
br.form['password-password-lt200'] = '@kYg9@-f7vfqJJJ'

# Login
br.submit()

<!DOCTYPE html>
<html dir="ltr" lang="en">
<head>
<title>The Best Mountain Biking Trails in Whistler | Trailforks</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="The best &amp; most popular mountain biking trails in Whistler. Trailforks ranks trails with a combination of user ratings and raw trail usage data from rides." name="description"/>
<meta content="https://es.pinkbike.org/246/sprt/i/trailforks/trailforks_avatar.png" property="og:image"/>
<meta content="Trailforks" property="og:site_name"/>
<meta content="158879950980488" property="fb:app_id"/>
<meta content="517713210" property="fb:admins"/>
<meta content="Trailforks" name="application-name"/>
<meta content="app-id=987986743" name="apple-itunes-app"/>
<meta content="#fec200" name="theme-color"/>
<link href="/manifest.json" rel="manifest"/>
<link href="//es.pinkbike.org" rel="dns-prefetch"/>
<link href="//ep1.pinkbik

In [79]:
titles = []
trails = []
site_nums = []

In [80]:
#total of 37277 pages
for i in range(1,37278):
    response = br.open('https://www.trailforks.com/tools/trailspreadsheet/?difficulty=5,6,8&cols=title,difficulty,rating,total_checkins,global_rank,distance,alias&rid={}'.format(i))
    html = response.read()
    response.close()
    page_soup = soup(html, 'lxml')
    num_trails = page_soup.find(id='contentTotal')
    if num_trails: 
        for string in num_trails.stripped_strings:
            num_trails = (string)
        title = page_soup.find('h3')
        for string in title.stripped_strings:
            title = string[:-13] #removing " Preview Data" from string
        titles.append(title)
        trails.append(num_trails)
        site_nums.append(i)
    else:
        continue
    wait = random.random()

    time.sleep(wait)

1
9
21
89
110
238
239
240
241
337
382
489
490
491
492
493
494
498
510
849
850
851
862
879
887
967
969
974
977
1011
1039
1040
1041
1042
1043
1044
1045
1047
1049
1081
1082
1083
1084
1329
1336
1352
1354
1460
1500
1503
1504
1505
1692
1693
1694
1695
1719
1728
1740
1743
1744
1844
1919
1949
1952
1958
1963
1964
1966
1983
1987
1988
1989
1990
1991
1993
2005
2183
2236
2338
2339
2342
2343
2440
2499
2558
2559
2562
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3018
3019
3020
3022
3023
3024
3025
3026
3027
3028
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3044
3047
3049
3051
3052
3053
3058
3059
3060
3088
3089
3091
3098
3101
3102
3106
3111
3113
3117
3125
3131
3136
3137
3138
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3171
3172
3173
3174
3175
3176
3177
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197


5032
5034
5035
5037
5038
5039
5040
5041
5042
5043
5045
5046
5047
5048
5049
5050
5051
5053
5055
5056
5057
5058
5060
5061
5062
5064
5065
5066
5068
5069
5070
5071
5074
5075
5076
5079
5081
5082
5083
5084
5085
5086
5087
5088
5090
5092
5095
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5115
5117
5118
5121
5122
5123
5124
5125
5126
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5147
5148
5150
5151
5152
5153
5155
5156
5157
5158
5159
5161
5162
5163
5164
5165
5166
5167
5168
5169
5171
5173
5175
5177
5178
5179
5180
5181
5182
5184
5187
5190
5192
5193
5196
5197
5198
5199
5200
5205
5206
5207
5208
5209
5210
5211
5212
5213
5215
5216
5217
5219
5221
5223
5224
5225
5229
5230
5231
5232
5233
5235
5237
5238
5239
5240
5241
5242
5243
5244
5246
5247
5250
5251
5252
5253
5254
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5283
5284
5285
5288
5289
5291
5292
5293
5294


7050
7051
7052
7053
7055
7056
7057
7058
7059
7060
7061
7062
7063
7066
7068
7069
7070
7073
7074
7075
7076
7077
7078
7080
7081
7082
7083
7084
7085
7087
7088
7089
7091
7092
7093
7094
7095
7096
7097
7098
7099
7101
7102
7105
7106
7108
7109
7110
7111
7113
7114
7116
7117
7118
7119
7120
7121
7122
7123
7126
7127
7128
7131
7132
7133
7134
7135
7137
7138
7139
7141
7142
7143
7144
7145
7146
7147
7148
7149
7150
7151
7152
7155
7156
7158
7160
7164
7165
7166
7167
7168
7169
7170
7171
7172
7173
7174
7175
7177
7178
7180
7181
7182
7183
7184
7185
7186
7187
7188
7189
7190
7191
7192
7194
7195
7196
7197
7198
7200
7203
7204
7206
7207
7209
7210
7211
7212
7213
7214
7216
7217
7218
7219
7221
7222
7224
7225
7226
7227
7228
7230
7231
7232
7235
7238
7239
7240
7241
7243
7244
7245
7248
7250
7251
7252
7253
7255
7256
7258
7259
7260
7262
7263
7264
7265
7266
7268
7269
7270
7271
7272
7274
7275
7277
7278
7279
7280
7281
7282
7284
7285
7286
7287
7289
7290
7291
7292
7293
7294
7295
7296
7297
7298
7299
7300
7301
7302
7304
7305
7306


9064
9065
9067
9072
9073
9076
9077
9078
9079
9080
9081
9082
9084
9085
9086
9087
9088
9089
9090
9091
9092
9093
9094
9095
9096
9097
9098
9099
9100
9101
9102
9103
9104
9105
9106
9107
9108
9109
9110
9111
9112
9113
9114
9115
9116
9117
9118
9119
9120
9121
9122
9123
9124
9125
9126
9127
9128
9129
9130
9131
9132
9134
9135
9137
9138
9140
9142
9143
9144
9145
9146
9147
9148
9150
9151
9152
9154
9155
9156
9157
9158
9159
9161
9163
9164
9165
9168
9169
9172
9173
9176
9177
9178
9179
9180
9181
9182
9183
9184
9185
9186
9187
9188
9189
9190
9191
9193
9194
9196
9197
9198
9199
9201
9202
9204
9205
9211
9212
9213
9215
9216
9217
9218
9219
9220
9221
9222
9223
9224
9225
9227
9228
9230
9231
9232
9234
9235
9236
9237
9238
9239
9242
9243
9246
9248
9249
9251
9252
9253
9254
9255
9257
9258
9259
9260
9261
9262
9263
9264
9265
9266
9267
9268
9269
9270
9271
9272
9275
9276
9277
9278
9279
9280
9281
9282
9283
9284
9285
9286
9287
9288
9289
9290
9292
9293
9294
9295
9297
9300
9301
9302
9303
9304
9305
9306
9307
9308
9309
9310
9311


10926
10928
10929
10930
10931
10932
10933
10934
10935
10936
10937
10938
10939
10940
10941
10943
10944
10945
10946
10947
10948
10949
10950
10951
10952
10953
10954
10955
10956
10957
10958
10959
10960
10961
10962
10963
10964
10965
10966
10967
10968
10969
10970
10971
10972
10973
10974
10975
10976
10977
10978
10979
10980
10981
10982
10983
10984
10985
10986
10987
10988
10989
10990
10991
10992
10993
10994
10995
10997
10998
10999
11000
11001
11002
11003
11004
11005
11006
11007
11008
11009
11010
11011
11012
11013
11014
11015
11016
11017
11018
11019
11020
11021
11024
11025
11027
11028
11029
11030
11031
11032
11033
11035
11036
11037
11038
11039
11040
11041
11042
11043
11044
11045
11046
11047
11048
11049
11050
11051
11052
11053
11054
11055
11056
11057
11058
11059
11060
11061
11062
11063
11064
11065
11066
11067
11068
11069
11070
11071
11072
11073
11074
11075
11076
11077
11078
11079
11080
11081
11082
11083
11084
11085
11087
11088
11089
11090
11091
11092
11094
11095
11096
11097
11098
11099
11100
1110

...

30887
30888
30889
30890
30891
30892
30893
30894
30895
30896
30897
30898
30899
30900
30901
30902
30903
30904
30905
30907
30908
30911
30916
30917
30918
30919
30920
30922
30923
30924
30925
30926
30927
30928
30929
30930
30931
30932
30933
30937
30938
30939
30942
30943
30944
30945
30946
30947
30948
30950
30951
30952
30953
30954
30955
30956
30958
30959
30961
30962
30963
30964
30966
30967
30969
30971
30972
30973
30974
30975
30976
30977
30978
30979
30980
30981
30982
30983
30984
30985
30986
30987
30988
30989
30990
30991
30992
30993
30994
30995
30997
30998
30999
31000
31002
31004
31005
31006
31008
31009
31010
31011
31012
31013
31014
31015
31016
31018
31019
31020
31021
31022
31023
31024
31025
31026
31027
31028
31029
31030
31031
31033
31034
31035
31036
31037
31038
31039
31040
31041
31042
31043
31044
31045
31046
31048
31049
31050
31051
31052
31053
31056
31057
31058
31059
31060
31061
31062
31063
31064
31066
31067
31068
31069
31070
31071
31072
31073
31075
31076
31077
31078
31079
31080
31082
31083
3108

32419
32420
32421
32424
32425
32426
32427
32429
32430
32431
32434
32435
32436
32437
32438
32440
32441
32442
32443
32444
32445
32446
32447
32448
32449
32450
32451
32452
32454
32455
32456
32457
32458
32459
32460
32461
32463
32464
32465
32466
32467
32468
32469
32470
32471
32473
32474
32475
32476
32477
32478
32479
32480
32481
32482
32483
32485
32486
32487
32488
32490
32491
32492
32493
32494
32496
32498
32499
32500
32501
32502
32503
32504
32505
32506
32507
32508
32509
32510
32511
32512
32514
32515
32516
32517
32518
32519
32520
32522
32523
32524
32525
32526
32527
32528
32529
32530
32531
32532
32533
32534
32535
32536
32537
32538
32539
32540
32541
32542
32543
32544
32545
32546
32547
32548
32549
32551
32552
32553
32555
32557
32558
32559
32560
32561
32562
32563
32564
32565
32566
32567
32568
32569
32570
32571
32573
32574
32575
32576
32577
32578
32580
32583
32584
32585
32586
32587
32588
32589
32591
32592
32593
32594
32595
32596
32597
32598
32599
32600
32601
32602
32603
32604
32605
32606
32607
3260

33953
33954
33955
33956
33958
33959
33960
33961
33962
33963
33964
33965
33966
33967
33968
33969
33970
33971
33972
33973
33974
33975
33976
33977
33978
33979
33980
33981
33982
33983
33984
33985
33986
33987
33988
33989
33990
33991
33992
33993
33994
33995
33996
33999
34000
34001
34002
34003
34004
34005
34006
34007
34008
34009
34010
34011
34012
34013
34014
34015
34016
34017
34018
34019
34020
34021
34022
34023
34024
34025
34026
34027
34028
34029
34030
34031
34032
34033
34034
34035
34036
34038
34039
34040
34041
34042
34043
34044
34045
34046
34047
34048
34049
34050
34051
34052
34053
34054
34055
34056
34057
34058
34059
34060
34061
34062
34063
34064
34065
34066
34067
34068
34069
34070
34071
34072
34073
34074
34075
34076
34077
34078
34079
34080
34081
34083
34084
34085
34088
34089
34090
34091
34092
34093
34094
34095
34096
34097
34098
34099
34100
34101
34102
34103
34104
34105
34106
34107
34108
34109
34110
34111
34112
34113
34114
34115
34116
34117
34118
34119
34120
34121
34122
34123
34124
34125
3412

35386
35387
35388
35389
35390
35391
35392
35393
35394
35395
35396
35397
35398
35399
35400
35401
35402
35403
35404
35405
35406
35407
35408
35409
35410
35411
35412
35413
35414
35415
35416
35417
35418
35419
35420
35421
35422
35423
35424
35425
35426
35427
35428
35429
35430
35431
35432
35433
35434
35435
35436
35439
35440
35441
35442
35443
35444
35445
35446
35447
35448
35449
35450
35451
35452
35453
35454
35455
35456
35457
35458
35459
35460
35461
35462
35463
35464
35465
35466
35467
35468
35469
35470
35471
35472
35473
35474
35475
35476
35477
35478
35479
35480
35481
35482
35483
35484
35485
35486
35487
35488
35489
35490
35491
35492
35493
35494
35495
35496
35497
35498
35499
35500
35501
35502
35503
35504
35505
35506
35507
35508
35509
35510
35511
35512
35513
35514
35515
35516
35517
35518
35519
35520
35521
35522
35523
35524
35525
35526
35527
35528
35529
35530
35531
35532
35533
35534
35535
35536
35537
35538
35539
35540
35541
35542
35543
35544
35545
35546
35547
35548
35549
35550
35551
35552
35553
3555

36930
36931
36932
36934
36935
36936
36937
36938
36939
36940
36941
36942
36943
36944
36945
36946
36947
36948
36949
36950
36951
36952
36953
36954
36955
36956
36957
36958
36959
36960
36961
36963
36966
36967
36968
36969
36970
36971
36972
36973
36975
36977
36978
36979
36981
36982
36983
36984
36985
36986
36987
36988
36989
36990
36991
36992
36993
36994
36995
36996
36997
36998
36999
37000
37001
37002
37003
37004
37005
37006
37007
37010
37011
37012
37013
37014
37015
37016
37017
37018
37019
37020
37021
37022
37023
37024
37025
37026
37027
37028
37029
37030
37031
37032
37033
37037
37038
37040
37041
37042
37043
37044
37046
37047
37048
37049
37050
37051
37052
37053
37054
37055
37056
37057
37058
37059
37060
37061
37062
37063
37064
37065
37066
37067
37068
37069
37070
37071
37072
37073
37074
37076
37077
37078
37079
37080
37081
37082
37083
37084
37085
37086
37087
37088
37089
37090
37091
37092
37093
37094
37095
37096
37097
37098
37099
37100
37101
37102
37103
37104
37105
37107
37108
37109
37110
37112
3711

Saving all data from scraper.  "trails" (amount of trails), "nums" (the directory num for trailforks.com), and "titles".

In [92]:
df = pd.DataFrame(titles, columns=["colummn"])
df.to_csv('titles.csv', index=False)
df1 = pd.DataFrame(trails, columns=["colummn"])
df1.to_csv('trails.csv', index=False)
df2 = pd.DataFrame(site_nums, columns=["colummn"])
df2.to_csv('site_nums.csv', index=False)

## More scraping
Postal codes for British Colombia

In [328]:
link ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V'

In [329]:
response = urlopen(link)
html = response.read()
response.close()

In [330]:
page_soup = soup(html, 'lxml')

In [331]:
tds = page_soup.find_all('td')

In [332]:
tds[1]

<td valign="top" width="11.1%"><b>V2A</b><br/><span style="font-size: smaller; line-height: 125%;"><a href="/wiki/Penticton" title="Penticton">Penticton</a></span>
</td>

In [377]:
postal_codes = []
city_names = []

In [378]:
for i in range(0,198):
    count = 0
    for string in tds[i].stripped_strings:
        if count == 0:
            postal_codes.append(string)
            count = 1
        elif count == 1:
            city_names.append(string)
            count = 2
        else:
            continue

Sanity check

In [379]:
len(postal_codes)

198

In [380]:
len(city_names)

198

## Loading Data

In [381]:
titles = pd.read_csv('titles.csv')
trails = pd.read_csv('trails.csv')

In [382]:
titles_list = titles.values.tolist()
titles_list = [item for sublist in titles_list for item in sublist]
titles_list

['Mount Fromme',
 'Mount Seymour',
 'Cypress Mountain',
 'Woodlot',
 'Burnaby Mountain / SFU',
 'Burke Mountain',
 'Eagle Mountain',
 'Bear Mountain',
 'Red Mountain',
 'Ledgeview',
 'Cumberland Forest',
 'Forbidden Plateau',
 'Valleycliffe',
 'Brackendale',
 'Alice Lake & Highlands',
 'Cat Lake',
 'Diamond Head',
 'China Ridge',
 'Sprockids',
 'Mosquito Lake',
 'Mackenzie',
 'One Mile Lake',
 'Vedder Mountain',
 'Whistler Mountain Bike Park',
 'Panorama Mountain Resort',
 'Sumas Mountain',
 'Delta Watershed',
 'Hope',
 'Bowen Island',
 'Duck Lake',
 'Sacred Mountain',
 'Hartland Mountain Bike Park',
 'Harbourview',
 'Burnt Bridge',
 'Mount Tzouhalem',
 'Maple Mountain',
 'Cobble Hill',
 'Mount Prevost',
 'Mount Washington',
 'Silver Star Mountain Resort',
 'Sun Peaks Resort',
 'Fernie Alpine Resort',
 'Kicking Horse Bike Park',
 'Top Bridge',
 'Doumont',
 'HammerFest',
 'The Abyss / Extension Ridge',
 'Coombs Candy',
 'Lost Lake',
 'Westside - Sproatt',
 'Whistler South',
 'Whistler N

In [383]:
trails_list = trails.values.tolist()
trails_list = [item for sublist in trails_list for item in sublist]
trails_list

['41',
 '29',
 '41',
 '13',
 '5',
 '26',
 '22',
 '6',
 '11',
 '28',
 '53',
 '47',
 '32',
 '0',
 '50',
 '0',
 '31',
 '0',
 '8',
 '21',
 '26',
 '10',
 '23',
 '59',
 '40',
 '9',
 '3',
 '4',
 '1',
 '6',
 '0',
 '39',
 '16',
 '12',
 '28',
 '4',
 '9',
 '24',
 '13',
 '18',
 '20',
 '19',
 '14',
 '0',
 '16',
 '11',
 '2',
 '9',
 '1',
 '37',
 '14',
 '26',
 '20',
 '2',
 '0',
 '4',
 '9',
 '4',
 '4',
 '3',
 '7',
 '20',
 '6',
 '0',
 '2',
 '8',
 '11',
 '8',
 '5',
 '17',
 '14',
 '9',
 '11',
 '1',
 '1',
 '3',
 '5',
 '1',
 '17',
 '2',
 '24',
 '6',
 '2',
 '3',
 '5',
 '17',
 '36',
 '18',
 '4,347',
 '11,174',
 '2,342',
 '747',
 '822',
 '416',
 '351',
 '478',
 '116',
 '136',
 '216',
 '134',
 '116',
 '45',
 '0',
 '77',
 '24',
 '78',
 '28',
 '62',
 '32',
 '15',
 '3',
 '40',
 '77',
 '24',
 '17',
 '39',
 '50',
 '204',
 '116',
 '54',
 '93',
 '67',
 '16',
 '72',
 '41',
 '42',
 '484',
 '48',
 '8',
 '31',
 '524',
 '623',
 '0',
 '47',
 '25',
 '19',
 '314',
 '3',
 '16',
 '1,323',
 '291',
 '15',
 '1,132',
 '5',
 '323',


Cleaning up the city names from the list.

In [384]:
for i in range(0,len(city_names)):
    string = city_names[i]
    num = string.find(' (')
    if num > 1:
        city_names[i] = string[:num]
    else:
        continue

## Combing Data
Matching the city names from BC postal codes to the search queries from Trailforks.com and creating a dataframe.

In [385]:
matched_cities = []
num_trails = []
postal_code = []

In [386]:
for i in range(0,len(city_names)):
    if city_names[i] in titles_list:
        matched_cities.append(city_names[i])
        index = titles_list.index(city_names[i])
        num_trails.append(trails_list[index])
        postal_code.append(postal_codes[i])
    else:
        #print("** NOT IN **", city_names[i])
        continue

Some popular mountain biking towns got left out, so I'm adding them manually.

In [387]:
matched_cities.append('Pemberton')
num_trails.append(77)
postal_code.append('V0N 2L0')
matched_cities.append('Gibsons')
num_trails.append(53)
postal_code.append('V0N 1V0')

In [388]:
dict = {'City': matched_cities, 'Number_of_Trails': num_trails, 'Postal_Code': postal_code}
df = pd.DataFrame(dict)

In [389]:
df['Number_of_Trails'] = df['Number_of_Trails'].astype(str).astype(int)

Dropping rows with 0 trails and also duplicates.

In [390]:
indexNames = df[ df['Number_of_Trails'] == 0 ].index

In [391]:
df.drop(indexNames, inplace=True)

In [392]:
df = df.reset_index(drop=True)

In [393]:
df = df = df.drop_duplicates(subset='City', keep='first')

In [394]:
df = df.reset_index(drop=True)

In [395]:
df

,City,Number_of_Trails,Postal_Code
0,Kimberley,15,V1A
1,Penticton,50,V2A
2,Surrey,14,V4A
3,Burnaby,5,V5A
4,Powell River,24,V8A
5,Victoria,72,V9A
6,Vernon,59,V1B
7,Kamloops,40,V2B
8,Squamish,116,V8B
9,Cranbrook,32,V1C


## Geospatial Coordinates

In [396]:
latitudes = []
longitudes = []

In [397]:
for i in range(0,len(df)):
    g=geocoder.arcgis('{}, British Colombia, Canada'.format(df['Postal_Code'][i]))
    lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    latitudes.append(latitude)
    longitudes.append(longitude)

In [398]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

Geocoder wasn't entirely accurate, so I ended up fixing a few of the cities by hand. It was fun.

In [402]:
df.at[6, 'Latitude'] = 50.268479
df.at[6, 'Longitude'] = -119.262568
df.at[7, 'Latitude'] = 50.678385
df.at[7, 'Longitude'] = -120.329835
df.at[8, 'Latitude'] = 49.700790
df.at[8, 'Longitude'] = -123.150588
df.at[13, 'Latitude'] = 49.285005
df.at[13, 'Longitude'] = -122.792470
df.at[14, 'Latitude'] = 50.115669
df.at[14, 'Longitude'] = -122.956799
df.at[15, 'Latitude'] = 52.140202
df.at[15, 'Longitude'] = -122.142379
df.at[16, 'Latitude'] = 49.049329
df.at[16, 'Longitude'] = -122.302905
df.at[21, 'Latitude'] = 50.031340
df.at[21, 'Longitude'] = -125.270810
df.at[25, 'Latitude'] = 50.109933
df.at[25, 'Longitude'] = -120.788705
df.at[26, 'Latitude'] = 53.915608
df.at[26, 'Longitude'] = -122.753380
df.at[28, 'Latitude'] = 49.491569
df.at[28, 'Longitude'] = -117.292110
df.at[29, 'Latitude'] = 48.780387
df.at[29, 'Longitude'] = -123.700198
df.at[32, 'Latitude'] = 49.879701
df.at[32, 'Longitude'] = -119.476071
df.at[33, 'Latitude'] = 49.154957
df.at[33, 'Longitude'] = -121.954321
df.at[33, 'Latitude'] = 49.154957
df.at[33, 'Longitude'] = -121.954321
df.at[42, 'Latitude'] = 48.378334
df.at[42, 'Longitude'] = -123.734941
df.at[46, 'Latitude'] = 49.400077
df.at[46, 'Longitude'] = -123.516955

In [524]:
df.head()

,City,Number_of_Trails,Postal_Code,Latitude,Longitude
0,Kimberley,15,V1A,49.691079,-115.952463
1,Penticton,50,V2A,49.491010,-119.574217
2,Surrey,14,V4A,49.032073,-122.821241
3,Burnaby,5,V5A,49.266244,-122.931096
4,Powell River,24,V8A,49.857390,-124.467470


In [310]:
g=geocoder.arcgis('British Colombia, Canada')
lat_lng_coords = g.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print('The geographic location of British Colombia, Canada is: Lat: {}, Lon: {}.'.format(latitude,longitude))

The geographic location of British Colombia, Canada is: Lat: 49.049279947775744, Lon: -122.29502008050079.


## Visualizing the data
I've scaled the map markers to show which towns have more or less trails.

In [404]:
# create map of BC using latitude and longitude values
map_bc = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, City, Number_of_Trails in zip(df['Latitude'], df['Longitude'], df['City'], df['Number_of_Trails']):
    label = '{}, {}'.format(City, Number_of_Trails)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        #Scaling the radius to the number of trails
        radius=np.sqrt(Number_of_Trails),
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bc)  
    
map_bc

## Foursquare API

In [361]:
CLIENT_ID = ''
CLIENT_SECRET = ''
VERSION = '20180605' # Foursquare API version

Checking the first city.

In [191]:
city_latitude = df.loc[0, 'Latitude'] # City latitude value
city_longitude = df.loc[0, 'Longitude'] # City longitude value

city_name = df.loc[0, 'City'] # City name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Kimberley are 49.69107910900004, -115.95246287899994.


In [362]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 3000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=EWPDNM354JQYFXVCVRQGM0ESHNAJPFWHQYVCNYTTFLMAY05Q&client_secret=KBC0ZQ43YYBGPU00T0WMGAUI3O3TJ2LP4TF0SBFGA5BLY1CE&v=20180605&ll=49.69107910900004,-115.95246287899994&radius=3000&limit=100'

In [363]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec2d45298205d001c4a87d9'},
 'response': {'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 49.71807913600007,
    'lng': -115.91080379723256},
   'sw': {'lat': 49.664079082000015, 'lng': -115.99412196076732}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4f778cabe4b0c7b6e208f425',
       'name': 'Pedal & Tap',
       'location': {'address': '215 Spokane St',
        'lat': 49.68589523177228,
        'lng': -115.98341795012075,
        'labeledLatLngs': [{'label': 'display',
          'lat': 49.68589523177228,
          'lng': -115.98341795012075}],
        'distance': 2302,
        'postalCode': 'V1A 2

In [364]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [365]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pedal & Tap,American Restaurant,49.685895,-115.983418
1,Shoppers Drug Mart,Pharmacy,49.684390,-115.981893
2,A&W,Fast Food Restaurant,49.684436,-115.981015
3,The Bean Tree Cafe,Café,49.685532,-115.982466
4,Platzl,Plaza,49.685953,-115.983600


In [366]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


Now lets get the results for all of the cities!

In [531]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [532]:
bc_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Kimberley
Penticton
Surrey
Burnaby
Powell River
Victoria
Vernon
Kamloops
Squamish
Cranbrook
Delta
Kitimat
Salmon Arm
Coquitlam
Whistler
Williams Lake
Abbotsford
North Vancouver
Terrace
Ladysmith
Port Moody
Campbell River
Fort St. John
Quesnel
Courtenay
Merritt
Prince George
Qualicum Beach
Nelson
Duncan
Comox
Castlegar
Kelowna
Chilliwack
Parksville
Trail
Maple Ridge
Nanaimo
Mission
West Vancouver
West Kelowna
Port Alberni
Sooke
North
Similkameen
Pemberton
Gibsons


In [533]:
bc_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Abbotsford,78,78,78,78,78,78
Burnaby,36,36,36,36,36,36
Campbell River,26,26,26,26,26,26
Chilliwack,78,78,78,78,78,78
Comox,4,4,4,4,4,4
Coquitlam,100,100,100,100,100,100
Cranbrook,3,3,3,3,3,3
Delta,29,29,29,29,29,29
Duncan,38,38,38,38,38,38


In [534]:
bc_venues.shape

(1195, 7)

In [535]:
bc_venues.head(10)

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kimberley,49.691079,-115.952463,BigHorn Electric,49.689229,-115.940580,Construction & Landscaping
1,Penticton,49.491010,-119.574217,Theo's Restaurant,49.493150,-119.589925,Restaurant
2,Penticton,49.491010,-119.574217,Earls,49.478106,-119.583512,Restaurant
3,Penticton,49.491010,-119.574217,Wild Scallion,49.501219,-119.592440,Vegetarian / Vegan Restaurant
4,Penticton,49.491010,-119.574217,The Copper Mug Pub,49.485062,-119.587979,Pub
5,Penticton,49.491010,-119.574217,The Pasta Factory,49.499574,-119.593845,Italian Restaurant
6,Penticton,49.491010,-119.574217,Penticton Farmer's Market,49.500265,-119.593121,Farmers Market
7,Penticton,49.491010,-119.574217,BRODO KITCHEN,49.495968,-119.591185,Soup Place
8,Penticton,49.491010,-119.574217,Cannery Brewing Co.,49.482953,-119.594019,Brewery
9,Penticton,49.491010,-119.574217,The Bench Market,49.502949,-119.586857,Coffee Shop


Adding the trails from Trailforks.com as 'venues'.

In [536]:
city_list = []
venue_list = []
for i in range(0, df.shape[0]):
    city = df['City'][i]
    iterations = df['Number_of_Trails'][i]
    for i in range(0,iterations):
        #y = pd.DataFrame([[city,'Trail']], columns=('City','Venue Category'))
        city_list.append(city)
        venue_list.append('Trail')
dict = {'City': city_list, 'Venue Category': venue_list}
df2 = pd.DataFrame(dict)
bc_venues_with_trails = bc_venues.append(df2, ignore_index=True)
bc_venues_with_trails

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kimberley,49.691079,-115.952463,BigHorn Electric,49.689229,-115.940580,Construction & Landscaping
1,Penticton,49.491010,-119.574217,Theo's Restaurant,49.493150,-119.589925,Restaurant
2,Penticton,49.491010,-119.574217,Earls,49.478106,-119.583512,Restaurant
3,Penticton,49.491010,-119.574217,Wild Scallion,49.501219,-119.592440,Vegetarian / Vegan Restaurant
4,Penticton,49.491010,-119.574217,The Copper Mug Pub,49.485062,-119.587979,Pub
...,...,...,...,...,...,...,...
2832,Gibsons,NaN,NaN,NaN,NaN,NaN,Trail
2833,Gibsons,NaN,NaN,NaN,NaN,NaN,Trail
2834,Gibsons,NaN,NaN,NaN,NaN,NaN,Trail
2835,Gibsons,NaN,NaN,NaN,NaN,NaN,Trail


In [537]:
print('There are {} uniques categories.'.format(len(bc_venues_with_trails['Venue Category'].unique())))

There are 187 uniques categories.


## Preparing Data for Clustering
Converting categorical data into integers so we can use them for clustering.

In [541]:
# one hot encoding
bc_onehot = pd.get_dummies(bc_venues_with_trails[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
bc_onehot['City'] = bc_venues_with_trails['City'] 

# move city column to the first column
fixed_columns = [bc_onehot.columns[-1]] + list(bc_onehot.columns[:-1])
bc_onehot = bc_onehot[fixed_columns]

bc_onehot.head(5)

,Yoga Studio,Airport,Airport Terminal,American Restaurant,Apres Ski Bar,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,...,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [542]:
bc_grouped = bc_onehot.groupby('City').mean().reset_index()
bc_grouped.head()

,City,Yoga Studio,Airport,Airport Terminal,American Restaurant,Apres Ski Bar,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Winery,Women's Store
0,Abbotsford,0.0,0.000,0.0,0.000000,0.0,0.008696,0.008696,0.0,0.0,...,0.0,0.0,0.008696,0.0,0.00000,0.0,0.0,0.0,0.0,0.008696
1,Burnaby,0.0,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.02439,0.0,0.0,0.0,0.0,0.000000
2,Campbell River,0.0,0.000,0.0,0.014925,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
3,Castlegar,0.0,0.000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000
4,Chilliwack,0.0,0.008,0.0,0.008000,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.008000,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000


Checking the top 5 venues from each city.  Obviously it will be skewed towards trails, but that's OK because it's by far the most important thing to me.

In [543]:
num_top_venues = 5

for city in bc_grouped['City']:
    print("----"+city+"----")
    temp = bc_grouped[bc_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Abbotsford----
                  venue  freq
0                 Trail  0.32
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.04
3      Sushi Restaurant  0.03
4              Pharmacy  0.03


----Burnaby----
            venue  freq
0           Trail  0.17
1            Café  0.07
2            Park  0.07
3  Sandwich Place  0.07
4    Burger Joint  0.07


----Campbell River----
                  venue  freq
0                 Trail  0.63
1  Fast Food Restaurant  0.04
2           Coffee Shop  0.04
3                  Bank  0.03
4              Pharmacy  0.03


----Castlegar----
           venue  freq
0          Trail   1.0
1    Yoga Studio   0.0
2          Motel   0.0
3  Movie Theater   0.0
4  Moving Target   0.0


----Chilliwack----
                  venue  freq
0                 Trail  0.38
1  Fast Food Restaurant  0.07
2           Coffee Shop  0.06
3            Restaurant  0.04
4        Sandwich Place  0.03


----Comox----
                 venue  freq
0                Trail  0.33
1

                   venue  freq
0                  Trail  0.69
1                  Hotel  0.04
2  Outdoors & Recreation  0.02
3                 Lounge  0.01
4      French Restaurant  0.01


----Williams Lake----
                           venue  freq
0                          Trail  0.74
1              Convenience Store  0.03
2                    Coffee Shop  0.03
3  Paper / Office Supplies Store  0.02
4                 Breakfast Spot  0.02




In [544]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [545]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = bc_grouped['City']

for ind in np.arange(bc_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bc_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Abbotsford,Trail,Coffee Shop,Fast Food Restaurant,Sandwich Place,Restaurant
1,Burnaby,Trail,Burger Joint,Sandwich Place,Café,Park
2,Campbell River,Trail,Fast Food Restaurant,Coffee Shop,Grocery Store,Pharmacy
3,Castlegar,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
4,Chilliwack,Trail,Fast Food Restaurant,Coffee Shop,Restaurant,Sandwich Place


# Clustering

In [546]:
# set number of clusters
kclusters = 3

bc_grouped_clustering = bc_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 1, 0, 0, 0, 1, 1, 0])

In [547]:
labels = kmeans.labels_.astype('int64')
labels.shape

(47,)

In [548]:
# add clustering labels
cities_venues_sorted.insert(0, 'Cluster Labels', labels.astype('int64'))

bc_merged = df

# merge bc_grouped with bc_data to add latitude/longitude for each city
bc_merged = bc_merged.join(cities_venues_sorted.set_index('City'), on='City')

In [549]:
bc_merged.head()

,City,Number_of_Trails,Postal_Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Kimberley,15,V1A,49.691079,-115.952463,1,Trail,Construction & Landscaping,Convenience Store,Cosmetics Shop,Fish & Chips Shop
1,Penticton,50,V2A,49.491010,-119.574217,2,Trail,Coffee Shop,Fast Food Restaurant,Grocery Store,Pizza Place
2,Surrey,14,V4A,49.032073,-122.821241,0,Trail,Coffee Shop,Café,Pizza Place,Japanese Restaurant
3,Burnaby,5,V5A,49.266244,-122.931096,0,Trail,Burger Joint,Sandwich Place,Café,Park
4,Powell River,24,V8A,49.857390,-124.467470,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


In [550]:
bc_merged = bc_merged.dropna(axis=0)
bc_merged['Cluster Labels'] = bc_merged['Cluster Labels'].astype(int)

In [551]:
bc_merged['Cluster Labels'][0:10]

0    1
1    2
2    0
3    0
4    1
5    2
6    0
7    0
8    2
9    1
Name: Cluster Labels, dtype: int64

## Visualize Clusters

In [552]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, nt in zip(bc_merged['Latitude'], bc_merged['Longitude'], bc_merged['City'], bc_merged['Cluster Labels'],bc_merged['Number_of_Trails']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) +' Trails: ' + str(nt), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=np.sqrt(nt),
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results

### Big city (purple)
Many of these locations seem to be in areas with bigger cities.

In [562]:
bc_merged.loc[bc_merged['Cluster Labels'] == 0, bc_merged.columns[[0]+[1] + list(range(5, bc_merged.shape[1]))]]

,City,Number_of_Trails,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Surrey,14,0,Trail,Coffee Shop,Café,Pizza Place,Japanese Restaurant
3,Burnaby,5,0,Trail,Burger Joint,Sandwich Place,Café,Park
6,Vernon,59,0,Trail,Coffee Shop,Fast Food Restaurant,Grocery Store,Pharmacy
7,Kamloops,40,0,Trail,Hotel,Restaurant,Coffee Shop,Bank
10,Delta,3,0,Trail,Discount Store,Breakfast Spot,Fast Food Restaurant,Pub
13,Coquitlam,60,0,Trail,Coffee Shop,Sushi Restaurant,Vietnamese Restaurant,Sandwich Place
16,Abbotsford,37,0,Trail,Coffee Shop,Fast Food Restaurant,Sandwich Place,Restaurant
19,Ladysmith,4,0,Trail,Construction & Landscaping,Harbor / Marina,Fast Food Restaurant,Hotel
20,Port Moody,3,0,Trail,Beach,Lake,Park,Dog Run
22,Fort St. John,5,0,Trail,Coffee Shop,Fast Food Restaurant,Pizza Place,Bank


### Fishing! (green)
It seems this cluster is dominated by areas with lot's of access to fishing.

In [563]:
bc_merged.loc[bc_merged['Cluster Labels'] == 1, bc_merged.columns[[0]+[1] + list(range(5, bc_merged.shape[1]))]]

,City,Number_of_Trails,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Kimberley,15,1,Trail,Construction & Landscaping,Convenience Store,Cosmetics Shop,Fish & Chips Shop
4,Powell River,24,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
9,Cranbrook,32,1,Trail,Construction & Landscaping,Home Service,Furniture / Home Store,Dog Run
12,Salmon Arm,11,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
17,North Vancouver,77,1,Trail,Café,Pizza Place,Taco Place,Gluten-free Restaurant
18,Terrace,16,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
23,Quesnel,7,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
24,Courtenay,1,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
31,Castlegar,6,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant
35,Trail,2,1,Trail,Fishing Spot,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant


### Rural Areas (red)
This cluster seems to be comprised of more rural cities/towns (with the big exception of Victoria).  I've spent a good deal of time in Whistler, Squamish, and Nelson (all of which are in this cluster), so that leads me to believe there might be other places from this cluster that I might like.  In fact, before starting this project, I was looking into Gibsons, and Duncan as potential places to live.

In [564]:
bc_merged.loc[bc_merged['Cluster Labels'] == 2, bc_merged.columns[[0]+[1] + list(range(5, bc_merged.shape[1]))]]

,City,Number_of_Trails,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Penticton,50,2,Trail,Coffee Shop,Fast Food Restaurant,Grocery Store,Pizza Place
5,Victoria,72,2,Trail,Coffee Shop,Restaurant,Grocery Store,Department Store
8,Squamish,116,2,Trail,Coffee Shop,Hotel,Scenic Lookout,Restaurant
11,Kitimat,3,2,Trail,Construction & Landscaping,Hotel,Beach,Women's Store
14,Whistler,216,2,Trail,Hotel,Outdoors & Recreation,Park,Plaza
15,Williams Lake,48,2,Trail,Convenience Store,Coffee Shop,Paper / Office Supplies Store,Liquor Store
21,Campbell River,41,2,Trail,Fast Food Restaurant,Coffee Shop,Grocery Store,Pharmacy
25,Merritt,24,2,Trail,Café,Convenience Store,Inn,Fast Food Restaurant
27,Qualicum Beach,3,2,Trail,Food Truck,Construction & Landscaping,Cosmetics Shop,Dry Cleaner
28,Nelson,46,2,Trail,Coffee Shop,Restaurant,Pub,Fast Food Restaurant
